In [1]:
import pandas as pd
import os
import io

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [70]:
def replace_blink_values(val):
    if val != 0:
        x = 1
    else:
        x = 0
    return x


def identificar_blink_eeg(val):
    try: 
        x = int(val)
        if x > 150:
            return 500
        else:
            return val
    except:
        return val

In [71]:
def read_txt_data(data_path, atividade):
    data_list = []
    with io.open(data_path, mode="r", encoding="utf-8") as f:
        for line in f:
            data_list.append(line.rstrip().split())
    df = pd.DataFrame(data_list)
    
    df.columns=df.iloc[0] 
    df = df.iloc[1:]
    df.iloc[-1,-1] = df.iloc[-1,3]


    df = df.loc[df["DATATYPE"] != "MSG"]
    df = df.loc[df["USER"].str.isdigit()]


    df.USER = df.USER.astype(int)
    df.BKID = df.BKID.astype(float)

    df["BKID"] = df.BKID.apply(replace_blink_values)
    df['USER'] = df.USER.apply(identificar_blink_eeg)

    df["EEG"] = df["USER"]
    
    
    df = df.reset_index()
    df = df.drop(["index","USER"],axis=1)
    df["task"] = atividade
    
    return df

In [72]:
df_happy = read_txt_data("/Users/anasouza/ppgeb_masters/data/emotions/happy_video.txt", "happy")
df_scary = read_txt_data("/Users/anasouza/ppgeb_masters/data/emotions/scary_video.txt", "scary")

In [73]:
df = df_happy.append(df_scary)

In [74]:
df = df.drop(["DATATYPE","CNT","TIME"], axis=1)

In [69]:
df

,LPCX,LPCY,LPD,LPS,LPV,RPCX,RPCY,RPD,RPS,RPV,...,REYEX,REYEY,REYEZ,RPUPILD,RPUPILV,BKID,BKDUR,BKPMIN,EEG,task
0,0.29490,0.16934,7.83348,1.14669,0,0.63348,0.16648,8.41400,1.14669,0,...,0.02357,0.03767,0.64804,0.00318,1,500,0.00000,30,51,happy
1,0.29490,0.16934,7.83348,1.14669,0,0.63348,0.16648,8.41400,1.14669,0,...,0.02357,0.03767,0.64804,0.00318,1,500,0.00000,30,51,happy
2,0.29490,0.16934,7.83348,1.14669,0,0.63348,0.16648,8.41400,1.14669,0,...,0.02357,0.03767,0.64804,0.00318,1,500,0.00000,30,51,happy
3,0.29490,0.16934,7.83348,1.14669,0,0.63348,0.16648,8.41400,1.14669,0,...,0.02357,0.03767,0.64804,0.00318,1,500,0.00000,30,50,happy
4,0.29490,0.16934,7.83348,1.14669,0,0.63348,0.16648,8.41400,1.14669,0,...,0.02357,0.03767,0.64804,0.00318,1,500,0.00000,30,50,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0.28800,0.13645,12.51273,1.11856,1,0.63393,0.11633,12.92266,1.10924,1,...,0.02324,0.04142,0.62140,0.00370,1,0,0.00000,36,64,scary
1082,0.28814,0.13637,12.54618,1.10924,1,0.63392,0.11640,12.95051,1.10924,1,...,0.02324,0.04142,0.62140,0.00364,1,0,0.00000,36,73,scary
1083,0.28810,0.13634,12.62192,1.10924,1,0.63395,0.11636,12.99150,1.10924,1,...,0.02324,0.04142,0.62140,0.00364,1,0,0.00000,36,55,scary
1084,0.28815,0.13616,12.67966,1.09945,1,0.63401,0.11626,12.51653,1.10877,1,...,0.02454,0.04356,0.65446,0.00366,1,0,0.00000,36,55,scary


In [75]:
# Tentando com Pycaret

In [76]:
!pip install --pre pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.5/544.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.3/538.3 kB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.7 MB/s eta 0:00:0000:0100:01
  Using cached lightgbm-3.3.2-py3-none-macosx_10_14_x86_64.macosx_10_15_x86_64.macosx_11_0_x86_64.whl (1.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 3.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.1 MB/s eta 0:00:0

In [77]:
from pycaret.classification import *

ImportError: cannot import name 'Iterable' from 'collections' (/Users/anasouza/opt/anaconda3/envs/python-sync/lib/python3.10/collections/__init__.py)

In [79]:
df.to_csv("/Users/anasouza/ppgeb_masters/data/emotion.csv")

In [80]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)

In [81]:
training_data.to_csv("/Users/anasouza/ppgeb_masters/data/emotion_train.csv")
testing_data.to_csv("/Users/anasouza/ppgeb_masters/data/emotion_test.csv")

In [13]:



def clean_dataset(df):
    
    df.USER = df.USER.replace("START_RECORDING", 100).replace("STOP_EYETRACKER", 200)
    df = df.loc[df["USER"] != "CLIENT2_READY"]
    df = df.loc[df.USER.notnull()]

    df.USER = df.USER.astype(int)
    df.BKID = df.BKID.astype(float)

    df["BKID"] = df.BKID.apply(replace_blink_values)
    df['USER'] = df.USER.apply(identificar_blink_eeg)

    df["BKID"] = df["BKID"].astype("object")
    df["USER"] = df["USER"].astype("object")
    df["USER"] = df["USER"].astype("object")

    eeg = df.iloc[:,[2,-1]]
    et = df.iloc[:,:-1]

    return df, eeg, et

In [14]:
df, eeg, et = clean_dataset(df)

/var/folders/c7/n7fskkg1029g6xhbfw1vpc8c0000gn/T/ipykernel_74049/941702302.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.USER = df.USER.replace("START_RECORDING", 100).replace("STOP_EYETRACKER", 200)


ValueError: invalid literal for int() with base 10: 'USER'